In [9]:
import datetime, json, csv, requests, pytz, math
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py

In [2]:
# load raw data from json
with open("txns.json", "r") as jf:
    txn_data = json.load(jf)
chains = list(txn_data.keys())
txns = []
# parse data into uniform dictionary
for chain in chains:
    for txn in txn_data[chain]:
        row = {x:txn[x] for x in txn.keys() if x not in ["from","to","timestamp"]}
        for prefix in ["from","to"]:
            for key in ["address","owner_type","owner"]:
                if key in txn[prefix].keys():
                    row["{p}_{k}".format(p=prefix,k=key)] = txn[prefix][key]
                else:
                    row["{p}_{k}".format(p=prefix,k=key)] = "unknown"
        row["timestamp"] = pd.to_datetime(txn["timestamp"], unit="s").replace(tzinfo=pytz.utc)
        txns.append(row)
            
df_txns = pd.DataFrame(data=txns)
df_txns.head()

,blockchain,symbol,id,transaction_type,hash,amount,amount_usd,transaction_count,from_address,from_owner_type,from_owner,to_address,to_owner_type,to_owner,timestamp
0,ethereum,usdt,905051047,transfer,e88e6b54ec47c4a1ad5815d2a7605d92ef9aee9ba30f08...,1.001703e+06,1007633.2,1,a5407eae9ba41422680e2e00537571bcc53efbfd,unknown,unknown,407bd75b50906c09a70692175abfdaa219921fa7,unknown,unknown,2020-08-24 20:02:10+00:00
1,ethereum,usdt,905051476,transfer,f10c141c74e9e21bb3eaa46ecf216f410b7cd603352418...,1.290730e+06,1298371.4,1,a5407eae9ba41422680e2e00537571bcc53efbfd,unknown,unknown,f094c56b95f52a23c0c280c293dd744179bad102,unknown,unknown,2020-08-24 20:03:18+00:00
2,ethereum,eth,905060463,transfer,0e803300dac7d24c6f5963abef3c4949f2c0b51283b270...,3.100000e+03,1254810.0,1,a577110d143bd1d1105a31e8fc19b3f65de3ebed,unknown,unknown,30e116751746619d37d72b50d158a249eeace14d,unknown,unknown,2020-08-24 20:07:20+00:00
3,ethereum,eth,905079261,transfer,11972d4054f2cd237640c3eeedf9bd6ebe91216e016a12...,1.942316e+03,784805.8,1,cfc22227f2325de43bd8772df5d5d50ec555469f,unknown,unknown,e5d3285e51357d9cd6f81cd7e94d6fd4299eec9b,unknown,unknown,2020-08-24 20:15:49+00:00
4,ethereum,eth,905080366,transfer,7953617a147a350d2e917a0eb9ea3bbf7e842e84d4af03...,2.000000e+03,802345.2,1,7b4576d06d0ce1f83f9a9b76bf8077bffd34fcb1,unknown,unknown,0b9de8705d125f10ab6885c3ce755f473b5cb897,unknown,unknown,2020-08-24 20:17:13+00:00


In [3]:
# visualize geo data
df_geo = pd.read_json("geos.json", orient="index")
df_geo

,location,lat,lon,address
binance,Malta,35.9375,14.3754,"85602 Melita Court, Level 3 Triq Giuseppe Cali..."
pax treasury,New York City,40.7128,-74.0060,"450 Lexington Avenue New York, NY 10017"
huobi,Singapore,1.3521,103.8198,"10 Anson Road, International Plaza, Singapore,..."
ftx,"San Francisco, CA",37.7749,-122.4194,"San Francisco, CA, United States"
kraken,"San Francisco, CA",37.7749,-122.4194,"548 Market Street Suite 39656 San Francisco, C..."
...,...,...,...,...
cobra1104 wallet,"San Francisco, CA",37.7749,-122.4194,"315 Montgomery St, San Francisco, CA, United S..."
chris larsen,"San Francisco, CA",37.7749,-122.4194,"315 Montgomery St, San Francisco, CA, United S..."
sdf ecosystem support - infrastructure grants,None,NaN,NaN,None
sdf operations,None,NaN,NaN,None


In [4]:
def appendGeoData(owners, key, geo_dict):
    series = []
    for owner in owners:
        series.append(geo_dict[owner][key])
    return series

# load geo data as json
with open("geos.json", "r") as jf:
    geo_dict = json.load(jf, encoding="utf8")
    
# append geo data to txn dataframe    
for key in ["location","lon","lat","address"]:
    from_series = appendGeoData(list(df_txns["from_owner"]), key, geo_dict)
    to_series = appendGeoData(list(df_txns["to_owner"]), key, geo_dict)
    df_txns["from_{}".format(key)] = from_series
    df_txns["to_{}".format(key)] = to_series
df_txns.head()

,blockchain,symbol,id,transaction_type,hash,amount,amount_usd,transaction_count,from_address,from_owner_type,...,to_address,to_owner_type,to_owner,timestamp,from_location,to_location,from_lon,to_lon,from_lat,to_lat
0,ethereum,usdt,905051047,transfer,e88e6b54ec47c4a1ad5815d2a7605d92ef9aee9ba30f08...,1.001703e+06,1007633.2,1,None,unknown,...,None,unknown,unknown,2020-08-24 20:02:10+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628
1,ethereum,usdt,905051476,transfer,f10c141c74e9e21bb3eaa46ecf216f410b7cd603352418...,1.290730e+06,1298371.4,1,None,unknown,...,None,unknown,unknown,2020-08-24 20:03:18+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628
2,ethereum,eth,905060463,transfer,0e803300dac7d24c6f5963abef3c4949f2c0b51283b270...,3.100000e+03,1254810.0,1,None,unknown,...,None,unknown,unknown,2020-08-24 20:07:20+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628
3,ethereum,eth,905079261,transfer,11972d4054f2cd237640c3eeedf9bd6ebe91216e016a12...,1.942316e+03,784805.8,1,None,unknown,...,None,unknown,unknown,2020-08-24 20:15:49+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628
4,ethereum,eth,905080366,transfer,7953617a147a350d2e917a0eb9ea3bbf7e842e84d4af03...,2.000000e+03,802345.2,1,None,unknown,...,None,unknown,unknown,2020-08-24 20:17:13+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628


In [5]:
def roundToDay(dt):
    return dt.strftime("%Y-%m-%d")
    #return datetime.datetime(dt.year, dt.month, dt.day).replace(tzinfo=pytz.utc)
df_txns["day"] = df_txns["timestamp"].apply(roundToDay)
df_txns.head()

,blockchain,symbol,id,transaction_type,hash,amount,amount_usd,transaction_count,from_address,from_owner_type,...,to_owner_type,to_owner,timestamp,from_location,to_location,from_lon,to_lon,from_lat,to_lat,day
0,ethereum,usdt,905051047,transfer,e88e6b54ec47c4a1ad5815d2a7605d92ef9aee9ba30f08...,1.001703e+06,1007633.2,1,None,unknown,...,unknown,unknown,2020-08-24 20:02:10+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628,2020-08-24
1,ethereum,usdt,905051476,transfer,f10c141c74e9e21bb3eaa46ecf216f410b7cd603352418...,1.290730e+06,1298371.4,1,None,unknown,...,unknown,unknown,2020-08-24 20:03:18+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628,2020-08-24
2,ethereum,eth,905060463,transfer,0e803300dac7d24c6f5963abef3c4949f2c0b51283b270...,3.100000e+03,1254810.0,1,None,unknown,...,unknown,unknown,2020-08-24 20:07:20+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628,2020-08-24
3,ethereum,eth,905079261,transfer,11972d4054f2cd237640c3eeedf9bd6ebe91216e016a12...,1.942316e+03,784805.8,1,None,unknown,...,unknown,unknown,2020-08-24 20:15:49+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628,2020-08-24
4,ethereum,eth,905080366,transfer,7953617a147a350d2e917a0eb9ea3bbf7e842e84d4af03...,2.000000e+03,802345.2,1,None,unknown,...,unknown,unknown,2020-08-24 20:17:13+00:00,Unknown,Unknown,135.0,135.0,-82.8628,-82.8628,2020-08-24


In [13]:
hover_data = {"symbol":True, "from_owner": True, "from_location": True, "to_owner": True, "to_location": True, "amount_usd": True, }
fig = px.scatter_geo(df_txns, "to_lat", "to_lon", color = "blockchain", size = "amount_usd", animation_frame = "day",
                     width = 1400, height = 800, projection = "natural earth", hover_data = hover_data, size_max = 40)
#pio.show(fig)

In [14]:
pio.write_html(fig, file="onchain_txn_map.html", auto_open=True)